# Async IO

In [ ]:
# pip install --upgrade pip aiohttp aiofiles

**Two keywords:** *async* and *await*

In [3]:
import asyncio
import time

## Asynchronous version

In [16]:
async def count(i):
    print(f"{i}: One")
    await asyncio.sleep(1)
    print(f"{i}: Two")

In [14]:
async def main():
    await asyncio.gather(count(1), count(2), count(3))

In [17]:
# Original code:
# -----------------------------
# s = time.perf_counter()
# asyncio.run(main())
# elapsed = time.perf_counter() - s
# print(f"Code executed in {elapsed:0.2f} seconds.")
# -----------------------------


# Handling for environments with an already running event loop
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError as ie:
    print(f'Error: {ie}')

s = time.perf_counter()

# Get the current event loop and run `main`
loop = asyncio.get_event_loop()

# Since I am using Jupyter, this will work fine
loop.run_until_complete(main())

elapsed = time.perf_counter() - s
print(f"Code executed in {elapsed:0.4f} seconds.")

1: One
2: One
3: One
1: Two
2: Two
3: Two
Code executed in 1.0033 seconds.


## Synchronous version

In [19]:
def count(i):
    print(f"{i}: One")
    time.sleep(1)
    print(f"{i}: Two")

def main():
    for i in range(3):
        count(i)

s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f"Code executed in {elapsed:0.2f} seconds.")

0: One
0: Two
1: One
1: Two
2: One
2: Two
Code executed in 3.01 seconds.


---------

In [2]:
import asyncio
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)


async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)

    # Here c[0] is used to Reset to default — so the rest of the terminal output remains unaffected.
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 3) for i in range(3)))
    return res


try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError as ie:
    print(f'Error: {ie}')

random.seed(444)
r1, r2, r3 = asyncio.run(main())
print()
print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0).
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2).
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
---> Finished: makerandom(1) == 7
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
---> Finished: makerandom(0) == 8
---> Finished: makerandom(2) == 10

r1: 8, r2: 7, r3: 10
